Script to scrape data from KenPom's website

Created: 12/15/2016

In [1]:
# Load packages for this script
import mechanize
from bs4 import BeautifulSoup
import urllib2 
import cookielib
import requests
import pandas as pd
import numpy as np
import re
import html5lib

In [84]:
def create_column_title(columns, year):
    """
    Function to create column titles for the data during scraping.
    """
    column_title = []
    first_line = columns[0].find_all('th')
    second_line = columns[1].find_all("th")
    if not second_line:
        for x in first_line:
            x_colspan = int(x['colspan']) if x.has_attr('colspan') else 1
            column_title.append(x.text)
            if x_colspan == 2:
                column_title.append(x.text + 'Rank')
    else:
        count_first = 0
        count_second = 0
        count = 0
        for x in first_line:
            count_first += int(x['colspan']) if x.has_attr('colspan') else 1
            while count_second < count_first:
                y = second_line[count]
                y_colspan = int(y['colspan']) if y.has_attr('colspan') else 1
                count_second += y_colspan
                column_title.append(x.text + y.text)
                if y_colspan == 2:
                    column_title.append(x.text + y.text + 'Rank')
                count += 1
    return column_title

# Create a method that parses a given year and spits out a raw dataframe
def import_raw_year(year):
    """
    Imports raw data from a ken pom year into a dataframe
    """
    f = s.get(url_year(year))
    soup = BeautifulSoup(f.text)
    table_html = soup.find_all('table', {'id': 'ratings-table'})

    thead = table_html[0].find_all('thead')
    columns = soup.find_all('tr')

    table = table_html[0]
    for x in thead:
        table = str(table).replace(str(x), '')

    df = pd.read_html(table)[0]
    df.columns = create_column_title(columns, year)
    df['Year'] = year
    df = df[df['Team'] == df['Team']]
    return df

# Base url, and a lambda func to return url for a given year
base_urls = ['http://kenpom.com/summary.php', \
             'http://kenpom.com/stats.php', \
             'http://kenpom.com/pointdist.php', \
             'http://kenpom.com/height.php', \
             'http://kenpom.com/teamstats.php']

# A dataframe for the data to be scraped
df_final = None

for base_url in base_urls:
    url_year = lambda x: '%s?y=%s' % (base_url, str(x))

    if base_url == 'http://kenpom.com/height.php':
        years = range(2007, 2018)
    elif base_url == 'http://kenpom.com/pointdist.php':
        years = range(2003, 2008)
    else:
        years = range(2002, 2018)

    # Login to website before scraping
    payload = { 'email': 'john.ezekowitz@gmail.com', 'password': 'STEEEEVE', 'submit': 'Login!'}
    with requests.Session() as s:
        p = s.post('http://kenpom.com/handlers/login_handler.php', data=payload)
        
    df = None
    for x in years:
        df = pd.merge(df, import_raw_year(x), how='outer') if df is not None else import_raw_year(years[0])

    df_final = pd.merge(df_final, df, how='outer', on=['Team','Year','Conf']) if df_final is not None else df
        
    # Column rename based off of original website
    #df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
    #             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
    #             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
    #             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
    #             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']

    # Lambda that returns true if given string is a number and a valid seed number (1-16)
    #valid_seed = lambda x: True if str(x).replace(' ', '').isdigit() \
    #                and int(x) > 0 and int(x) <= 16 else False

    # Use lambda to parse out seed/team
    #df['Seed'] = df['Team'].apply(lambda x: x[-2:].replace(' ', '') \
    #                              if valid_seed(x[-2:]) else np.nan )

    #df['Team'] = df['Team'].apply(lambda x: x[:-2] if valid_seed(x[-2:]) else x)

    # Split W-L column into wins and losses
    #df['Wins'] = df['W-L'].apply(lambda x: int(re.sub('-.*', '', x)) )
    #df['Losses'] = df['W-L'].apply(lambda x: int(re.sub('.*-', '', x)) )
    #df.drop('W-L', inplace=True, axis=1)


In [115]:
# Post-processing on the final dataframe
df_final = df_final.dropna(axis=1)
df_final = df_final[df_final['Team'] != 'Team']

In [119]:
df_final[df_final['Year'] == 2002]

,Team,Conf,TempoAdjusted,TempoAdjustedRank,TempoRaw,TempoRawRank,Year,AdjTempo,AdjTempoRank,OffenseAdjOE,...,OffenseOR%,OffenseOR%Rank,OffenseFTRate,3P%,3P%Rank,2P%,2P%Rank,FT%,FT%Rank,Blk%
0,Alabama A&M,SWAC,79.9,1,81.6,1,2002.0,79.9,1,90.2,...,38.3,41,35.4,31.6,272,44.3,284,66.4,253,8.3
1,VMI,SC,78.6,2,79.7,3,2002.0,78.6,2,97.2,...,33.3,187,30.6,32.8,228,46.1,231,70.3,117,9.4
2,Kansas 1,B12,78.4,3,78.6,5,2002.0,78.4,3,116.7,...,41.2,4,35.2,41.1,3,52.6,23,71.9,72,9.1
3,TCU,CUSA,78.3,4,79.1,4,2002.0,78.3,4,107.8,...,34.7,138,36.6,34.1,175,46.4,226,71.0,94,9.6
4,Arkansas Pine Bluff,SWAC,77.2,5,80.2,2,2002.0,77.2,5,80.2,...,29.5,288,34.0,27.6,322,40.1,324,65.9,262,9.0
5,Grambling St.,SWAC,76.2,6,78.5,6,2002.0,76.2,6,95.2,...,34.4,146,32.6,34.7,150,47.1,188,67.2,227,6.6
6,Prairie View A&M,SWAC,76.1,7,78.4,7,2002.0,76.1,7,87.6,...,34.4,147,38.2,32.4,240,46.8,205,64.5,285,8.9
7,LIU Brooklyn,NEC,76.0,8,76.6,10,2002.0,76.0,8,95.4,...,30.0,280,29.9,30.4,298,49.2,111,68.4,188,5.5
8,Duke 1,ACC,75.9,9,76.8,9,2002.0,75.9,9,118.9,...,34.5,144,41.0,36.3,86,57.4,2,69.0,164,7.5
9,Gardner Webb,ind,75.8,10,76.6,11,2002.0,75.8,10,99.2,...,32.4,219,35.5,35.7,106,47.6,169,67.3,222,8.8
